#### Author: Alkistis Pourtsidou, QMUL

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [ ]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

In [ ]:
import scipy
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy import integrate as integrate
from __future__ import division

In [ ]:
pi=np.pi

In [ ]:
#plots settings
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'
plt.rcParams['text.usetex']=True

In [ ]:
#Fiducial cosmological parameters
c=3e5
hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

In [ ]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [ ]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [ ]:
#Get matter power spectrum at z=0: P(k,z=0)
pars.set_matter_power(redshifts=[0.], kmax=100.0)
#Non-Linear spectra (Halofit)
pars.NonLinear = model.NonLinear_both
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-6, maxkh=100.0, npoints = 1000)

In [ ]:
#Construct P(k,z=0) interpolating function
Pkz0 = interp1d(kh, pk[0])

In [ ]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00
#Define the comoving distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

In [ ]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

In [ ]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

In [ ]:
#fiducial bHI 
bHI = 1.0

In [ ]:
#choose redshift
zbin = 0.5

In [ ]:
#Construct matter P(k,z); no RSD
def Pkz(kk):    
    return pow(Dgz(zbin),2)*Pkz0(kk)

In [ ]:
#Construct observed P_ell(k) with toy foreground removal and telescope beam
#note brightness temperature not included so units are just Mpc^3/h^3

R = 5 #beam smoothing scale [Mpc/h]

def dP0(mu):
    beta = fg(zbin)/bHI
    return pow(1+beta*mu**2,2)

def P0(kk,mu1):#mu1 is used for foreground removal effects toy model
    return (1/2.)*pow(bHI,2)*Pkz(kk)*scipy.special.dawsn(kk*R)/(kk*R)*(scipy.integrate.romberg(dP0,-1,-mu1)\
                                                                             +scipy.integrate.romberg(dP0,mu1,1))
def dP2(mu):
    beta = fg(zbin)/bHI
    return pow(1+beta*mu**2,2)*(3*mu**2-1)/2

def P2(kk,mu1):#mu1 is used for foreground removal effects toy model 
    return (5/2.)*pow(bHI,2)*Pkz(kk)*scipy.special.dawsn(kk*R)/(kk*R)*(scipy.integrate.romberg(dP2,-1,-mu1)\
                                                                             +scipy.integrate.romberg(dP2,mu1,1))

def dP4(mu):
    beta = fg(zbin)/bHI
    return pow(1+beta*mu**2,2)*(35*mu**4-30*mu**2+3)/8

def P4(kk,mu1):#mu1 is used for foreground removal effects toy model
    return (9/2.)*pow(bHI,2)*Pkz(kk)*scipy.special.dawsn(kk*R)/(kk*R)*(scipy.integrate.romberg(dP4,-1,-mu1)\
                                                                             +scipy.integrate.romberg(dP4,mu1,1))

#### FOR TESTING ####

#analytical P0 formula (without beam)
def P0anal(kk):
    beta = fg(zbin)/bHI
    return (1 + (2/3)*beta + (1/5)*beta**2)*bHI**2*Pkz(kk)

#analytical P2 formula (without beam)
def P2anal(kk):
    beta = fg(zbin)/bHI
    return ((4/3)*beta + (4/7)*beta**2)*bHI**2*Pkz(kk)

In [ ]:
kminn=3e-3
kmaxx=0.3

ktab = np.linspace(kminn,kmaxx,400)

itab = [0,1]

lstab = ['-', '--']
colortab = ['black','red']

mu1tab = [0,0.1] #0 means perfect foreground cleaning!
legtab = ['no FG','with FG']


for i in itab:
    p0 = [P0(k,mu1tab[i]) for k in ktab]
    plt.plot(ktab,ktab*p0,color=colortab[i],ls=lstab[i],label=legtab[i])

#plt.xlim(kminn,kmaxx)
plt.ylim(0,700)

plt.xlabel("$k \, [h \, \mathrm{Mpc^{-1}]}$")
plt.ylabel("$k P_0(k) \, \mathrm{[Mpc^2/h^2]}$")
plt.legend()

#plt.savefig('P0.pdf', format="pdf", bbox_inches = 'tight')

plt.show()

In [ ]:
for i in itab:
    p2 = [P2(k,mu1tab[i]) for k in ktab]
    plt.plot(ktab,ktab*p2,color=colortab[i],ls=lstab[i],label=legtab[i])

plt.xlabel("$k \, [h \, \mathrm{Mpc^{-1}]}$")
plt.ylabel("$k P_2(k) \, \mathrm{[Mpc^2/h^2]}$")
plt.legend()

plt.ylim(0,700)

#plt.savefig('P2.pdf', format="pdf", bbox_inches = 'tight')

plt.show()

In [ ]:
for i in itab:
    p4 = [P4(k,mu1tab[i]) for k in ktab]
    plt.plot(ktab,ktab*p4,color=colortab[i],ls=lstab[i],label=legtab[i])

plt.xlabel("$k \, [h \, \mathrm{Mpc^{-1}]}$")
plt.ylabel("$k P_4(k) \, \mathrm{[Mpc^2/h^2]}$")
plt.legend()

plt.ylim(-100,100)

#plt.savefig('P4.pdf', format="pdf", bbox_inches = 'tight')

plt.show()

#### February 2019